In [6]:
from dotenv import load_dotenv
import os
load_dotenv(override=True)
from typing import Annotated, Optional, TypedDict, List
from langchain_community.document_loaders.web_base import WebBaseLoader
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)


class DebateState(TypedDict):
    topic: str
    argument_a: str
    argument_b: str
    verdict: str



def debater_a(state: DebateState, llm):
    prompt = f"""
You are Debater A.
Argue **FOR** the topic below with strong logical reasoning.

Topic: {state['topic']}
"""
    response = llm.invoke([HumanMessage(content=prompt)])
    return {"argument_a": response.content}


def debater_b(state: DebateState, llm):
    prompt = f"""
You are Debater B.
Argue **AGAINST** the topic below with strong counterpoints.

Topic: {state['topic']}
"""
    response = llm.invoke([HumanMessage(content=prompt)])
    return {"argument_b": response.content}

def judge(state: DebateState, llm):
    prompt = f"""
You are a neutral judge.

Topic: {state['topic']}

Argument FOR:
{state['argument_a']}

Argument AGAINST:
{state['argument_b']}

Decide the winner.
Return:
Winner: Debater A or Debater B
Reason: concise explanation
"""
    response = llm.invoke([HumanMessage(content=prompt)])
    return {"verdict": response.content}


graph = StateGraph(DebateState)

graph.add_node("debater_a", lambda s: debater_a(s, llm))
graph.add_node("debater_b", lambda s: debater_b(s, llm))
graph.add_node("judge", lambda s: judge(s, llm))

graph.add_edge(START, "debater_a")
graph.add_edge("debater_a", "debater_b")
graph.add_edge("debater_b", "judge")
graph.add_edge("judge", END)

app = graph.compile()


result = app.invoke({
    "topic": "AI will replace most software engineers in the next 10 years"
})

print("🅰️ Argument A:\n", result["argument_a"])
print("\n🅱️ Argument B:\n", result["argument_b"])
print("\n⚖️ Verdict:\n", result["verdict"])

🅰️ Argument A:
 **Argument For: AI Will Replace Most Software Engineers in the Next 10 Years**

1. **Rapid Advancements in AI Technology**: The pace of AI development has accelerated dramatically in recent years. With advancements in machine learning, natural language processing, and automated coding tools, AI systems are becoming increasingly capable of performing complex tasks that were once the exclusive domain of human software engineers. Tools like GitHub Copilot and OpenAI's Codex are already demonstrating the ability to write code, debug, and even optimize software solutions, suggesting that the capabilities of AI will only continue to improve.

2. **Cost Efficiency and Productivity**: Companies are always looking for ways to reduce costs and increase productivity. AI can automate repetitive tasks, streamline workflows, and reduce the time required for software development. By replacing human engineers with AI, organizations can save on salaries, benefits, and training costs whi